### **GPU cehck**

In [1]:
import tensorflow as tf
tf.test.is_built_with_cuda()
tf.config.list_physical_devices('GPU')
tf.sysconfig.get_build_info()

2022-01-10 07:21:30.560963: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-10 07:21:52.805566: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-10 07:21:52.807943: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-10 07:21:52.861931: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:83:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-01-10 07:21:52.861976: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-01-10 07:21:53.099155: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2

OrderedDict([('cpu_compiler',
              '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614534878484/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'),
             ('cuda_compute_capabilities',
              ['compute_35',
               'compute_52',
               'compute_60',
               'compute_61',
               'compute_70',
               'compute_75']),
             ('cuda_version', '10.1'),
             ('cudnn_version', '7'),
             ('is_cuda_build', True),
             ('is_rocm_build', False)])

so.10
2022-01-10 07:21:53.505071: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2022-01-10 07:21:53.674582: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2022-01-10 07:21:53.817902: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.10
2022-01-10 07:21:54.182371: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-10 07:21:54.184811: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1862] Adding visible gpu devices: 0


## **Module**

In [133]:
from datetime import datetime
import os
import re
import gc
import pickle

import pandas as pd
import numpy as np
import seaborn as sns
from functools import reduce

from sklearn.manifold import Isomap
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import umap.umap_ as umap

# keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

## **UDF**

In [3]:
def cancer_select(cols, cancer_type):
    # phenotype
    phe1 = pd.read_csv("https://gdc-hub.s3.us-east-1.amazonaws.com/download/GDC-PANCAN.basic_phenotype.tsv.gz", sep="\t")
    phe1 = phe1.loc[phe1.program == "TCGA", :].loc[:, ['sample', 'sample_type', 'project_id']].drop_duplicates(['sample'])
    phe1['sample'] =  phe1.apply(lambda x : x['sample'][:-1], axis=1)
    phe2 = pd.read_csv("https://tcga-pancan-atlas-hub.s3.us-east-1.amazonaws.com/download/TCGA_phenotype_denseDataOnlyDownload.tsv.gz", sep="\t")
    ph_join = pd.merge(left = phe2 , right = phe1, how = "left", on = "sample").dropna(subset=['project_id'])
    
    if cancer_type == "PAN" or cancer_type == "PANCAN":
        filterd = ph_join.loc[ph_join['sample_type_y'] == "Primary Tumor", :]
        sample_barcode = filterd["sample"].tolist()
    else:
        filterd = ph_join.loc[(ph_join['sample_type_y'] == "Primary Tumor") & (ph_join['project_id'] == "TCGA-" + cancer_type) , :]
        sample_barcode = filterd["sample"].tolist()
        
    intersect_ = list(set(cols).intersection(sample_barcode))
    
    return intersect_

def non_zero_column(DF):
    sample_cnt = int(len(DF.columns) * 0.2)
    zero_row = dict(DF.isin([0]).sum(axis=1))
    non_remove_feature = list()

    for key, value in zero_row.items():
        if value < sample_cnt:
            non_remove_feature.append(key)
    
    return non_remove_feature


def determine_outlier_thresholds_iqr(dataframe, col_name, th1=0.05, th3=0.95):
    quartile1 = dataframe[col_name].quantile(th1)
    quartile3 = dataframe[col_name].quantile(th3)
    iqr = quartile3 - quartile1
    upper_limit = quartile3 + 1.5 * iqr
    lower_limit = quartile1 - 1.5 * iqr
    return lower_limit, upper_limit

def check_outliers_iqr(dataframe, col_name):
    lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)].any(axis=None):
        return True
    else: 
        return False
    
def replace_with_thresholds_iqr(dataframe, cols, th1=0.05, th3=0.95, replace=False):
    from tabulate import tabulate
    data = []
    for col_name in cols:
        if col_name != 'Outcome':
            outliers_ = check_outliers_iqr(dataframe,col_name)
            count = None
            lower_limit, upper_limit = determine_outlier_thresholds_iqr(dataframe, col_name, th1, th3)
            if outliers_:
                count = dataframe[(dataframe[col_name] > upper_limit) | (dataframe[col_name] < lower_limit)][col_name].count()
                if replace: 
                    dataframe.loc[(dataframe[col_name] < lower_limit), col_name] = lower_limit
                    dataframe.loc[(dataframe[col_name] > upper_limit), col_name] = upper_limit
            outliers_status = check_outliers_iqr(dataframe, col_name)
            data.append([outliers_, outliers_status, count, col_name, lower_limit, upper_limit ])
    table = tabulate(data, headers=['Outliers (Previously)', 'Outliers', 'Count', 'Column', 'Lower Limit', 'Upper Limit'], tablefmt='rst', numalign='right')
   
    return dataframe


def get_dataset(path, cancer_type, std):
    # non_scale dataset
    # load raw DF
    DF = pd.read_csv(path + "/feature/input_feature_non_scale/" + cancer_type + ".txt", sep = "\t")
    pr_sample_DF = DF.iloc[:, 0:1]

    if cancer_type == "PANCAN":
        DF = DF.drop(["Sample_barcode"], axis=1)
    else :
        DF = DF.drop(["Project", "Sample_barcode"], axis=1)
        
    # remove outlier
    DF = replace_with_thresholds_iqr(dataframe=DF, cols=DF.columns)
    max_number_of_nas = DF.shape[0] / 2
    DF = DF.loc[:, (DF.isnull().sum(axis=0) <= max_number_of_nas)]

    # normalization
    if std:
        scalerX = StandardScaler()
        scalerX.fit(DF)
        DF_sacle = scalerX.transform(DF)
    else :
        scalerX = MinMaxScaler()
        scalerX.fit(X_train)
        DF_sacle = scalerX.transform(DF)
    
    # missing impute
    imputer = KNNImputer(n_neighbors=10)
    DF_sacle_impute = imputer.fit_transform(DF_sacle)

    DF_sacle_impute = pd.DataFrame(DF_sacle_impute, columns=DF.columns)

    return DF_sacle_impute

# z = z_mean + sqrt(var) * epsilon
def sampling(args):
    """Reparameterization trick by sampling from an isotropic unit Gaussian.
    # Arguments
        args (tensor): mean and log of variance of Q(z|X)
    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean = 0 and std = 1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def get_train_test_DF(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, shuffle=True, random_state=32, stratify=y)
    
    return X_train, X_test, y_train, y_test

def createFolder(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print ('Folder already exists. ' +  path)

## **Preprocessing**

### **File Path**

In [4]:
os.getcwd()

'/home/km/gitworking/Multi-omics-intergration/src'

In [5]:
RAW_file_path = "/home/km/gitworking/Multi-omics-intergration/RAW_DATA/"
PKL_file_path = "/home/km/gitworking/Multi-omics-intergration/pkl/"
MODEL_path = "/home/km/gitworking/Multi-omics-intergration/models/"

* **RNA gene expression**

In [123]:
col = pd.read_csv(RAW_file_path + "tcga_RSEM_Hugo_norm_count.gz",
                 sep = "\t", index_col=0, nrows=0).columns.to_list()
use_col = ['sample'] + cancer_select(cols=col, cancer_type="LIHC")

In [124]:
df_chunk = pd.read_csv(RAW_file_path + "tcga_RSEM_Hugo_norm_count.gz",
                 sep = "\t", index_col=0, iterator=True, chunksize=50000, usecols=use_col)
rna = pd.concat([chunk for chunk in df_chunk])
rna = rna[rna.index.isin(non_zero_column(rna))].T

* **miRNA**

In [125]:
col = pd.read_csv(RAW_file_path + "pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.xena.gz",
                 sep = "\t", index_col=0, nrows=0).columns.to_list()
use_col = ['sample'] + cancer_select(cols=col, cancer_type="LIHC")

In [126]:
df_chunk = pd.read_csv(RAW_file_path + "pancanMiRs_EBadjOnProtocolPlatformWithoutRepsWithUnCorrectMiRs_08_04_16.xena.gz",
                 sep = "\t", index_col=0, iterator=True, chunksize=50000, usecols=use_col)
mirna = pd.concat([chunk for chunk in df_chunk])
mirna = mirna[mirna.index.isin(non_zero_column(mirna))].T

* **Methylation**

In [ ]:
col = pd.read_csv(RAW_file_path + "jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv.synapse_download_5096262.xena.gz",
                 sep = "\t", index_col=0, nrows=0).columns.to_list()
use_col = ['sample'] + cancer_select(cols=col, cancer_type="LIHC")

In [ ]:
df_chunk = pd.read_csv(RAW_file_path + "jhu-usc.edu_PANCAN_HumanMethylation450.betaValue_whitelisted.tsv.synapse_download_5096262.xena.gz",
                 sep = "\t", index_col=0, iterator=True, chunksize=50000, usecols=use_col)
mt = pd.concat([chunk for chunk in df_chunk])

In [13]:
mt_map = pd.read_csv(RAW_file_path + "probeMap_illuminaMethyl450_hg19_GPL16304_TCGAlegacy", sep="\t")

In [80]:
mt_join = pd.merge(mt, mt_map, how = "left", left_on = "sample", right_on = "#id")\
         .drop(['chrom', 'chromStart', 'chromEnd', 'strand', '#id'], axis=1)
mt_join = mt_join[mt_join.gene != "."]
mt_join.dropna(subset = ["gene"], inplace=True)

# gene mean 
mt_join_gene_filter = mt_join.groupby(['gene']).mean()
mt_join_gene_filter = mt_join_gene_filter[mt_join_gene_filter.index.isin(non_zero_column(mt_join_gene_filter))].T

In [127]:
rna['sample'] = rna.index

In [128]:
mirna['sample'] = mirna.index

In [129]:
mt_join_gene_filter['sample'] = mt_join_gene_filter.index

In [132]:
merge_list = [rna, mirna, mt_join_gene_filter]

In [134]:
omics = reduce(lambda left, right : pd.merge(left, right, on = "sample"), merge_list)

In [137]:
omics.set_index('sample', inplace=True)

In [140]:
omics

,RP11-16E12.2_x,RP11-181G12.2,CPHL1P_x,TCOF1_x,NSRP1_x,RP5-1000K24.2,SPPL3_x,AF001548.5,OPA3_x,OPA1_x,...,"chr22-38_28785274-29006793.1,ZNRF3-AS1,ZNRF3",pRNA,"pRNA,AP001347.6",pk_y,"pk,MLK7-AS1","snoR1,SNHG19,SNORD60,TRAF7","snoU13,KIF9","snoU2_19,EIF1AX,snoU2-30","uc_338,SMARCA4",yR211F11.2
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-CC-A3MA-01,3.0170,4.7785,0.0000,11.5952,10.4224,3.6565,10.3479,5.6826,9.9501,10.5289,...,0.226746,0.253931,0.802173,0.262715,0.841819,0.040540,0.901541,0.877858,0.951853,0.918813
TCGA-DD-AAW0-01,2.1896,2.6651,2.8547,10.4419,9.7246,3.4323,10.1216,4.5154,10.0724,11.9185,...,0.281292,0.353733,0.934289,0.274041,0.767279,0.035665,0.236923,0.429889,0.905745,0.931620
TCGA-DD-AACA-01,3.2386,2.1751,0.7682,10.7915,10.3504,2.8736,10.0403,4.0645,9.6837,11.4174,...,0.245278,0.149060,0.506923,0.247678,0.726268,0.032140,0.235719,0.392821,0.932847,0.312001
TCGA-K7-AAU7-01,5.5060,5.9382,7.3385,11.0277,10.0970,3.3956,11.3990,5.0913,10.6333,11.1081,...,0.255657,0.300331,0.639749,0.258850,0.776524,0.044826,0.120922,0.850816,0.929263,0.828439
TCGA-RC-A7SH-01,6.6832,4.9831,5.7378,10.8955,9.0594,0.9913,11.3930,6.7918,10.7746,10.6830,...,0.258233,0.408436,0.924842,0.246913,0.794186,0.046974,0.198767,0.889381,0.901486,0.876262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-2Y-A9H8-01,4.2129,6.1754,3.5807,10.4468,9.1961,2.6966,10.4525,3.5117,10.1593,11.2977,...,0.338572,0.376402,0.962969,0.290296,0.783476,0.034968,0.840098,0.825663,0.923310,0.893791
TCGA-2Y-A9GT-01,4.8918,5.7623,1.2425,9.7209,9.1073,3.8739,10.4595,5.1868,10.3863,10.7661,...,0.313733,0.369372,0.546060,0.311700,0.807100,0.045246,0.182878,0.939182,0.899394,0.609828
TCGA-DD-A73C-01,0.9754,3.7613,1.5521,9.8084,9.8115,1.5521,10.4740,3.0522,10.4710,12.4116,...,0.355588,0.377962,0.930226,0.269236,0.841078,0.036144,0.067261,0.942604,0.923079,0.798672
